In [1]:
from typing import Optional

from tqdm import trange
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.experimental import disable_dynamic_shapes
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.utils import coalesce, degree, cumsum
from torch_geometric import EdgeIndex

import pathpyG as pp
pp.config['torch']['device'] = 'cuda'

In [2]:
# Example with walks as node sequences
g = pp.Graph.from_edge_list([('a', 'b'), ('b', 'c'), ('a', 'c')])
dags = pp.DAGData(mapping = g.mapping)

dags.append_walk(('a', 'b', 'c'), weight=1.0)
dags.append_walk(('a', 'c'), weight = 2.0)
print(dags)

DAGData with 2 dags with total weight 3.0


In [3]:
# Example with walks as edge indices (with no mapping)
dags = pp.DAGData()
dags.append_dag(torch.tensor([[3,0,1],[0,1,2]]))
dags.append_dag(torch.tensor([[1,0,2],[0,2,0]]))
dags.append_dag(torch.tensor([[0,1],[1,2]]))
print(dags)

DAGData with 3 dags with total weight 3.0


In [4]:
# Example with mix of walks or dags
dags = pp.DAGData(mapping = g.mapping)

dags.append_dag(torch.tensor([[0,0,1],[1,2,2]]))
dags.append_walk(('a', 'b', 'c'))
print(dags)

DAGData with 2 dags with total weight 2.0


In [5]:
# Real-world example
dags = pp.DAGData.from_ngram('../data/tube_paths_train.ngram')
print(dags)

DAGData with 61748 dags with total weight 2147865.0


In [6]:
m = pp.MultiOrderModel.from_DAGs(dags, max_order=10)

In [7]:
print(m.layers[10])

Directed graph with 7053 nodes and 7473 edges

Node attributes
	node_sequences		<class 'torch.Tensor'> -> torch.Size([7053, 10])

Edge attributes
	edge_weights		<class 'torch.Tensor'> -> torch.Size([7473])

Graph attributes
	num_nodes		<class 'int'>



In [ ]:
pp.plot(m.layers[10], node_label=list(map(str, m.layers[1].data.node_sequences.tolist())))

In [8]:
dags.map_node_seq(m.layers[10].data.node_sequences[5].tolist())

['Southwark',
 'Waterloo',
 'Westminster',
 'Green Park',
 'Bond Street',
 'Baker Street',
 'Finchley Road',
 'HarrowOnTheHill',
 'Moor Park',
 'Northwood']

In [9]:
print(m.layers[2].data.edge_index)

EdgeIndex([[  0,   0,   0,  ..., 644, 644, 645],
           [  3,   4,   5,  ..., 504, 505, 582]], device='cuda:0',
          sparse_size=(646, 646), nnz=1139, sort_order=row)


In [10]:
print(m.layers[2].data.edge_weights)

tensor([ 279., 3875.,   45.,  ...,    5.,    5.,    5.], device='cuda:0')


In [11]:
print(m.layers[2].data.node_sequences)

tensor([[  0,   1],
        [  0, 115],
        [  1,   0],
        ...,
        [265, 130],
        [266, 194],
        [266, 235]], device='cuda:0')
